In [34]:
!pip install -qU langchain-huggingface


[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [42]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [43]:
from langchain_milvus import Milvus

URI = "./test.db"

vector_store = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": URI},
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [44]:
from langchain_core.documents import Document

vector_store_saved = Milvus.from_documents(
    [Document(page_content="foo!")],
    embeddings,
    collection_name="langchain_example",
    connection_args={"uri": URI},
)

In [45]:
vector_store_loaded = Milvus(
    embeddings,
    connection_args={"uri": URI},
    collection_name="langchain_example",
)

In [46]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['1d683090-5f62-42b5-a0e9-87bf363ed931',
 '6b99c318-e486-48f5-be4f-05bebd95a2c7',
 'df1a248c-a5de-4fb0-b7f9-9c780a731e2d',
 '2fd8599e-00e4-4343-bf73-492deed2f9b4',
 '3238e693-4a3d-435e-86c9-dfd7b8c31db2',
 '3133f00f-adb6-4076-a7b8-2bb26eae2876',
 '15293eec-ce70-445a-be57-f5dafc070ffd',
 '807c8924-0f0a-43b1-9d8a-b94b433e3189',
 'dfc3235c-d07e-406e-b80a-4722f80eb9cd',
 '4a90062e-550b-4993-a32a-005f3ffccbc8']

In [47]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Building an exciting new project with LangChain - come check it out! [{'pk': 'df1a248c-a5de-4fb0-b7f9-9c780a731e2d', 'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'pk': '807c8924-0f0a-43b1-9d8a-b94b433e3189', 'source': 'tweet'}]


In [48]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.809472] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'pk': '6b99c318-e486-48f5-be4f-05bebd95a2c7', 'source': 'news'}]


In [53]:
milvus_retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 1})
milvus_retriever.invoke("Stealing from the bank is a crime", filter={"source": "news"})

[Document(metadata={'pk': '2fd8599e-00e4-4343-bf73-492deed2f9b4', 'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.')]

In [56]:
from langchain_community.llms import Ollama
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

mistral_llm = Ollama(
        base_url="http://llm.ic-shared-llm.svc.cluster.local:11434",
        model="mistral",
        top_p=0.92,
        temperature=0.01,
        num_predict=512,
        repeat_penalty=1.03,
        callbacks=[StreamingStdOutCallbackHandler()]
    )

In [59]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


retriever = milvus_retriever
llm = mistral_llm

system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)

# chain.invoke({"input": "Hello"})